In [1]:
dbutils.library.installPyPI("mlflow")
dbutils.library.restartPython()
import mlflow

Wed Sep 9 08:32:56 2020 py4j imported

In [2]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [3]:
# reading parquet files from the curated data
dirPath = '/mnt/root/COVID19_TWEETS/v3/REFINED/WITH_SENTIMENT/'

In [4]:
# read the data in batch
tweets = spark.read.parquet(dirPath)

In [5]:
# just to get a feel of how much data we are dealing with:
tweets.where(tweets['user_location'] =='US').count()

Out[3]: 47

In [6]:
tweets.where(tweets["text"]=='').count()

Out[4]: 0

In [7]:
tweets.select("user_location").distinct().count()

Out[5]: 31503

In [8]:
tweets = tweets.where(tweets["text"]!='')

In [9]:
tweets.where(tweets["text"].isNull()).count()

Out[7]: 0

In [10]:
display(tweets.limit(10))

#Leveraging Spark ML:
Data wrangling, feature exctraction and mormalization:

Tuning with pyspark.ml in databricks environment, metrics and models are automatically logged to MLflow.

In [12]:
# arrange the data to fit to our needs:
from pyspark.sql.types import DoubleType
# drop null values
tweets = tweets.dropna(subset=["positive_sentiment","negative_sentiment","is_retweet","user_location","hashtags"])
#cast values
tweets = tweets.withColumn("negative_sentiment",tweets['negative_sentiment'].cast(DoubleType()))
tweets = tweets.withColumn("positive_sentiment",tweets['positive_sentiment'].cast(DoubleType()))
# fill null values with default value
tweets = tweets.fillna(0.0, subset=["user_followers","user_friends","user_favourites"])
tweets = tweets.fillna(False, subset=["user_verified"])

In [13]:


from pyspark.ml.classification import DecisionTreeClassifier, DecisionTreeClassificationModel
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.mllib.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer

prediction = None
#start mlflow model for tracking the experiment
with mlflow.start_run():
    #create experiment name
    mlflow.create_experiment("/retweets-sentiment-based-decision-tree")
    mlflow.spark.autolog
    # Read input columns location and hashtags - and annotate them as categorical values
    indexLocation = StringIndexer(inputCol="user_location",outputCol="indexedLocations").setHandleInvalid("keep")
    indexHashtages = StringIndexer(inputCol="hashtags",outputCol="indexedhashtags").setHandleInvalid("keep")

    # assemble multiple columns into one feature column, we often use this with many Spark mllib out of the box machine learnig models
    assembler = VectorAssembler(inputCols=[ "indexedLocations","indexedhashtags","user_followers", "user_friends","user_favourites","positive_sentiment","negative_sentiment","user_verified"], outputCol="features")

    # StringIndexer: Read input column "is_retweet" (true/false) and annotate them as categorical values.
    indexLabel = StringIndexer(inputCol="is_retweet", outputCol="indexedRetweetLabel").setHandleInvalid("keep")

    # most often with decision tree we would want to normalize the data to make it fit into the bins:
    normalizer = Normalizer(inputCol="features", outputCol="normFeatures", p=1.0)

    # DecisionTreeClassifier: Learn to predict column "indexedRetweetLabel" using the "normFeatures" column.
    dtc = DecisionTreeClassifier(labelCol="indexedRetweetLabel",featuresCol="normFeatures")

    # Chain indexer + dtc together into a single ML Pipeline.
    pipeline = Pipeline(stages=[indexLocation,indexHashtages,assembler,indexLabel,normalizer, dtc])

    # train the model
    model = pipeline.fit(tweets)
    # use the model - only for demo, for other cases we will use multiple datasets for testing and training
    prediction = model.transform(tweets)
    
    #path to persiste the model to
    model_path = "/mnt/root/COVID19_TWEETS/ML-Models/V1"
    # register the model with ml flow as well
    mlflow.register_model("retweet-sentiment-decision-tree","retweet-sentiment-based-decision-tree-v1")
    # persist the model to disk
    model.save(model_path)

In [14]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedRetweetLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(prediction)
print("Test Error = %g " % (1.0 - accuracy))

Test Error = 0.000368164

In [15]:
prediction.printSchema()

root
-- user_name: string (nullable = true)
-- user_location: string (nullable = true)
-- user_description: string (nullable = true)
-- user_created: string (nullable = true)
-- user_followers: float (nullable = false)
-- user_friends: float (nullable = false)
-- user_favourites: float (nullable = false)
-- user_verified: boolean (nullable = false)
-- date: string (nullable = true)
-- text: string (nullable = true)
-- hashtags: string (nullable = true)
-- source: string (nullable = true)
-- positive_sentiment: double (nullable = true)
-- negative_sentiment: double (nullable = true)
-- is_retweet: string (nullable = true)
-- indexedLocations: double (nullable = false)
-- indexedhashtags: double (nullable = false)
-- features: vector (nullable = true)
-- indexedRetweetLabel: double (nullable = false)
-- normFeatures: vector (nullable = true)
-- rawPrediction: vector (nullable = true)
-- probability: vector (nullable = true)
-- prediction: double (nullable = false)

In [16]:
selected = prediction.select("is_retweet","user_location","hashtags","positive_sentiment","prediction")
for row in selected.collect():
    is_retweet,user_location, hashtags,positive_sentiment,prediction = row
    print("is_retweet=%s , user_location=%s , positive_sentiment=%s , hashtags=%s --> prediction=%f" % (str(is_retweet),str(user_location),str(positive_sentiment) ,str(hashtags), prediction))
    


--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-1689419707369110> in <module> 
 ----> 1 selected = prediction . select ( "is_retweet" , "user_location" , "hashtags" , "positive_sentiment" , "prediction" ) 
 2 for row in selected . collect ( ) : 
 3 is_retweet , user_location , hashtags , positive_sentiment , prediction = row
 4 print ( "is_retweet=%s , user_location=%s , positive_sentiment=%s , hashtags=%s --> prediction=%f" % ( str ( is_retweet ) , str ( user_location ) , str ( positive_sentiment ) , str ( hashtags ) , prediction ) ) 
 5 

 AttributeError : 'float' object has no attribute 'select'